In [1]:
from dask import dataframe as dd 
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import datetime

In [14]:
def sql_reading(sysname_value):
    from sqlalchemy import create_engine
    
    """ Connect to the PostgreSQL database server """

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    engine = create_engine()
    df = pd.read_sql(
        "SELECT sysname, str_id, esm_v, datetime FROM battery_string WHERE sysname ='"+sysname_value+"' and datetime between '06/23/2021 07:00 AM' and '06/23/2021 08:50 AM';", engine) 
    print(sysname_value+' is analyzed.')
    return df

def grouping_v(df):
    conditions = [ 
        (df.esm_v<680),
        (df.esm_v>=680) & (df.esm_v<730),
        (df.esm_v>=730) & (df.esm_v<780),
        (df.esm_v>=780) & (df.esm_v<830),
        (df.esm_v>=830)
    ]
    values = [0,1,2,3,4]
    df2=pd.DataFrame()
    df2['datetime'] = df['datetime']
    df2['sysname'] = df['sysname']
    df2['str_id'] = df['str_id']
    df2['group_v'] = np.select(conditions,values)
    df2['esm_v'] = df['esm_v']
    print('Size: ',len(df2))
    return df2

def counting(df):
    df2 = pd.DataFrame()
    df2 = df[['sysname', 'str_id','group_v','datetime']].groupby(['sysname', 'str_id','group_v']).size().reset_index(name='count')
    return df2

def datetime_converting(df):
    df['datetimep'] = pd.to_datetime(df.datetime, format='%m/%d/%Y %H:%M', errors='coerce')
    mask = df.datetimep.isnull()
    df.loc[mask, 'datetimep'] = pd.to_datetime(df[mask]['datetime'], format='%Y-%m-%d %H:%M:%S.%f',
                                                 errors='coerce')
    df = df.drop(columns='datetime')
    df.rename(columns = {'datetimep':'datetime'}, inplace = True)
    df = df.sort_values(by=['str_id','sysname','group_v'])
    return df

def csv_saving(df,sysname):
    import os
    outdir = './Groups_csv_files'
    outname = 'esm_string_analysis_'+str(sysname)+'.csv'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    df.to_csv(fullname)
   

In [2]:
def counting_groups():
    import pyforest
    sysname_value = ''
    df_read = pd.DataFrame()
    df_counted = pd.DataFrame()
    df_saving = pd.DataFrame()
    sysname_arr = [
        'AE01','Andie','BK01','BK02','BS01','CC01','CR01','CR02','CR03','CR04','CR05','CS01','CS02',
        'CV01','CW01','GC01','GC02','HC01','HE01','HE02','HH01','HH02','HH03','HH04','HH05','HH06','HH07','HH08','HH09',
        'HH11','PY01','PY02','PY03','RD01','RD02','RD04','RD05','RD06','RD07','RD08','RD09','RD10','RD11','RD12','RD13',
        'RD14','RD15','RD16','RD17','SP01','SU01','SU02','SU03','Tammy']
    for i in range(len(sysname_arr)):
        sysname_value = sysname_arr[i]
        df_read = sql_reading(sysname_value)
        df_grouped = grouping_v(df_read)
        df_counted = counting(df_grouped)
        csv_saving(df_counted,sysname_value)
        print('CSV file of <',sysname_value,'> is ready!')
    return df_counted
    

In [14]:
def duration_calculating(url):
    df = pd.DataFrame()
    import_libs()
    df_read = reading_files(url)#example: "csv_string/esm_string_*.csv"
    df_grouped = grouping_v(df_read)
    df_duration = datetime_converting(df_grouped)
    return df_duration

In [10]:
import pyforest
import time
df = pd.DataFrame()
start = time.time()
df=counting_groups()
end = time.time()
print(end - start)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Connecting to the PostgreSQL database...


...

size:  881


<IPython.core.display.Javascript object>

CSV file of  BK01  is ready!
Connecting to the PostgreSQL database...


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [24]:
df

""


In [126]:
import pyforest
df = pd.DataFrame()
start = time.time()
df=counting_groups()#example: "csv_string/esm_string_*.csv"
end = time.time()
print(end - start)

0.0


In [103]:
import pyforest
df2 = pd.DataFrame()
start = time.time()
df2=duration_calculating('csv_string/esm_string_*.csv')#example: "csv_string/esm_string_*.csv"
end = time.time()
print(end - start)

141.26461362838745


In [104]:
df

,sysname,str_id,group_v,count
0,CW01,0,3,186
1,CW01,0,4,131128
2,CW01,1,3,4595
3,CW01,1,4,151230
4,CW01,2,3,3739
5,CW01,2,4,152150
6,CW01,3,3,5013
7,CW01,3,4,150839
8,CW01,4,3,4844
9,CW01,4,4,151159


In [116]:
pd.set_option('display.max_rows',100)
df2.head(100)

,sysname,str_id,group_v,esm_v,datetime
14661,CW01,0,3,839.9,2020-04-01 17:49:00
14668,CW01,0,3,839.9,2020-04-01 17:51:00
14672,CW01,0,3,839.9,2020-04-01 17:52:00
14679,CW01,0,3,839.9,2020-04-01 17:54:00
14686,CW01,0,3,839.9,2020-04-01 17:56:00
14693,CW01,0,3,839.9,2020-04-01 17:58:00
14700,CW01,0,3,839.9,2020-04-01 18:00:00
14707,CW01,0,3,839.8,2020-04-01 18:02:00
14714,CW01,0,3,839.8,2020-04-01 18:04:00
14724,CW01,0,3,839.8,2020-04-01 18:07:00


In [82]:
for i in df.datetime:
    if df.iloc[0]
    df['duration'] = (df.iloc[df.count.iloc['count'].datetime - df.iloc[].datetime).astype('timedelta64[h]')

IndexError: single positional indexer is out-of-bounds

In [72]:
from dask import dataframe as dd 
import pandas as pd
import numpy as np
import time
from glob import glob
#import glob 
import os

In [2]:
start = time.time()

#path = 'csv_string/'
#all_files = glob.glob(path + "/*.csv")
all_files = sorted(glob("csv_string/esm_string_*.csv"))


frame = dd.concat([dd.read_csv(f, delimiter = ',',encoding = 'UTF-8') for f in all_files], ignore_index = True)
#frame = pd.concat((pd.read_csv(f).assign(filename=f) 
         #          for f in all_files), ignore_index = True)
df = pd.DataFrame()
df = frame.compute()
end = time.time()
print(end - start)

40.722724199295044


In [3]:
start = time.time()

#path = 'csv_string/'
#all_files = glob.glob(path + "/*.csv")
all_files = sorted(glob("csv_string/esm_string_*.csv"))


frame = dd.concat([dd.read_csv(f, delimiter = ',',encoding = 'UTF-8') for f in all_files], ignore_index = True)
#frame = dd.concat((pd.read_csv(f,delimiter = '\t',encoding = 'UTF-8').assign(filename=f) 
                  # for f in all_files), ignore_index = True)
df = pd.DataFrame()
df = frame.compute()
end = time.time()
print(end - start)

44.74672341346741


In [4]:
df.head()

,Unnamed: 0,datetime,comname,sysname,str_id,status,soc,esm_v,curr,mincellv,...,temp0,temp1,temp2,maxtemp,mintemp,avgtemp,esmc_flags,stop0,stop1,stop2
0,1,12/19/2020 2:09,ChunWo,CW01,2,5,95.5,863.1,-0.21,4.042,...,29.0,27.0,27.3,29.0,27.0,27.8,15,2,439,65515
1,2,12/19/2020 2:09,ChunWo,CW01,4,517,100.0,862.8,-0.28,4.087,...,27.1,26.3,25.9,27.1,25.9,26.4,15,2,436,65508
2,3,12/19/2020 2:09,ChunWo,CW01,6,517,99.7,862.6,-0.24,4.087,...,27.6,26.1,26.2,27.6,26.1,26.6,15,2,434,65512
3,4,12/19/2020 2:10,ChunWo,CW01,0,517,99.3,863.5,0.37,4.091,...,27.1,26.1,25.9,27.1,25.9,26.4,15,2,443,37
4,5,12/19/2020 2:10,ChunWo,CW01,2,5,95.5,863.8,0.41,4.043,...,29.0,27.0,27.3,29.0,27.0,27.8,15,2,446,41


In [2]:
import time
from glob import glob
start = time.time()
#path = 'csv_string/'
#all_files = glob.glob(path + "/*.csv")
all_files = sorted(glob("csv_string/esm_string_*.csv"))


#frame = dd.concat([dd.read_csv(f, delimiter = '\t',encoding = 'UTF-8') for f in all_files], ignore_index = True)
frame = pd.concat((pd.read_csv(f,delimiter = ',',encoding = 'UTF-8').assign(filename=f) 
                   for f in all_files), ignore_index = True)

end = time.time()
print(end - start)

60.444647550582886


In [3]:
df = frame

In [4]:
df.groupby(['sysname','str_id'])

In [32]:
df

,Unnamed: 0,datetime,comname,sysname,str_id,status,soc,esm_v,curr,mincellv,...,temp1,temp2,maxtemp,mintemp,avgtemp,esmc_flags,stop0,stop1,stop2,filename
0,1,12/19/2020 2:09,ChunWo,CW01,2,5,95.5,863.1,-0.21,4.042,...,27.0,27.3,29.0,27.0,27.8,15,2,439,65515,csv_string\esm_string_cw01.csv
1,2,12/19/2020 2:09,ChunWo,CW01,4,517,100.0,862.8,-0.28,4.087,...,26.3,25.9,27.1,25.9,26.4,15,2,436,65508,csv_string\esm_string_cw01.csv
2,3,12/19/2020 2:09,ChunWo,CW01,6,517,99.7,862.6,-0.24,4.087,...,26.1,26.2,27.6,26.1,26.6,15,2,434,65512,csv_string\esm_string_cw01.csv
3,4,12/19/2020 2:10,ChunWo,CW01,0,517,99.3,863.5,0.37,4.091,...,26.1,25.9,27.1,25.9,26.4,15,2,443,37,csv_string\esm_string_cw01.csv
4,5,12/19/2020 2:10,ChunWo,CW01,2,5,95.5,863.8,0.41,4.043,...,27.0,27.3,29.0,27.0,27.8,15,2,446,41,csv_string\esm_string_cw01.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13469772,1958268,2021-04-21 09:19:01,Gammon,RD02,5,5,64.4,776.6,3.79,3.680,...,28.7,28.7,29.0,28.7,28.8,15,7766,379,98,csv_string\esm_string_rd02.csv
13469773,1958269,2021-04-21 09:20:00,Gammon,RD02,0,517,64.4,776.3,2.72,3.686,...,28.7,28.6,29.2,28.6,28.8,15,7763,272,187,csv_string\esm_string_rd02.csv
13469774,1958270,2021-04-21 09:20:01,Gammon,RD02,2,5,64.9,776.4,2.54,3.685,...,28.8,29.0,29.1,28.8,29.0,15,7764,254,126,csv_string\esm_string_rd02.csv
13469775,1958271,2021-04-21 09:20:01,Gammon,RD02,4,517,44.7,777.0,2.59,3.612,...,28.2,28.5,29.6,28.2,28.8,15,7770,259,118,csv_string\esm_string_rd02.csv


In [6]:
# iterate over all files within "My_Folder"
start = time.time()
for file in os.listdir("csv_string"):
    if file.endswith(".csv"):
        tmp = pd.read_csv(os.path.join("csv_string", file))
end = time.time()
print(end - start)

132.04864192008972


In [5]:
len(df)

13469777

In [6]:
start = time.time()
df.drop_duplicates()
end = time.time()
print(end - start)

83.95662641525269


In [18]:
start = time.time()
conditions = [ 
        (df.esm_v<680),
        (df.esm_v>=680) & (df.esm_v<730),
        (df.esm_v>=730) & (df.esm_v<780),
        (df.esm_v>=780) & (df.esm_v<830),
        (df.esm_v>=830)
    ]
values = [0,1,2,3,4]
df2=pd.DataFrame()
df2['datetime'] = df['datetime']
df2['sysname'] = df['sysname']
df2['str_id'] = df['str_id']
df2['group_v'] = np.select(conditions,values)
df2['esm_v'] = df['esm_v']
df2.head(150)
end = time.time()
print(end - start)

1.8332955837249756


In [35]:
df2.groupby(['sysname','str_id'])

In [36]:
df2.head()

,sysname,str_id,group_v,esm_v,datetime
0,CW01,2,4,863.1,2020-12-19 02:09:00
1,CW01,4,4,862.8,2020-12-19 02:09:00
2,CW01,6,4,862.6,2020-12-19 02:09:00
3,CW01,0,4,863.5,2020-12-19 02:10:00
4,CW01,2,4,863.8,2020-12-19 02:10:00


In [9]:
for i in range(0,5):
    a=len(df2[df2.group_v==i])
    print(i,':',a)

0 : 9
1 : 5444
2 : 260037
3 : 1019837
4 : 12184450


In [10]:
df2.groupby(['sysname','str_id','group_v'])

In [12]:
df2.head()

,datetime,sysname,str_id,group_v,esm_v
0,12/19/2020 2:09,CW01,2,4,863.1
1,12/19/2020 2:09,CW01,4,4,862.8
2,12/19/2020 2:09,CW01,6,4,862.6
3,12/19/2020 2:10,CW01,0,4,863.5
4,12/19/2020 2:10,CW01,2,4,863.8


In [198]:
df2.sort_values(by='datetime')

,datetime,sysname,str_id,group_v,esm_v
124420,1/1/2021 0:00,CW01,1,4,863.3
124422,1/1/2021 0:00,CW01,5,4,862.8
124421,1/1/2021 0:00,CW01,3,4,863.1
124423,1/1/2021 0:01,CW01,0,4,862.3
124424,1/1/2021 0:01,CW01,2,4,862.6
...,...,...,...,...,...
1048570,8/1/2020 9:39,CW01,6,4,849.9
1048571,8/1/2020 9:40,CW01,0,4,850.2
1048572,8/1/2020 9:40,CW01,2,4,850.2
1048573,8/1/2020 9:40,CW01,4,4,850.1


In [199]:
df2

,datetime,sysname,str_id,group_v,esm_v
0,12/19/2020 2:09,CW01,2,4,863.1
1,12/19/2020 2:09,CW01,4,4,862.8
2,12/19/2020 2:09,CW01,6,4,862.6
3,12/19/2020 2:10,CW01,0,4,863.5
4,12/19/2020 2:10,CW01,2,4,863.8
...,...,...,...,...,...
13469772,2021-04-21 09:19:01,RD02,5,2,776.6
13469773,2021-04-21 09:20:00,RD02,0,2,776.3
13469774,2021-04-21 09:20:01,RD02,2,2,776.4
13469775,2021-04-21 09:20:01,RD02,4,2,777.0


In [20]:
df2 = datetime_converting(df2)

In [21]:
pd.set_option('display.max_rows',1000)
df2

,sysname,str_id,group_v,esm_v,datetime
0,CW01,2,4,863.1,2020-12-19 02:09:00
1,CW01,4,4,862.8,2020-12-19 02:09:00
2,CW01,6,4,862.6,2020-12-19 02:09:00
3,CW01,0,4,863.5,2020-12-19 02:10:00
4,CW01,2,4,863.8,2020-12-19 02:10:00
...,...,...,...,...,...
13469772,RD02,5,2,776.6,2021-04-21 09:19:01
13469773,RD02,0,2,776.3,2021-04-21 09:20:00
13469774,RD02,2,2,776.4,2021-04-21 09:20:01
13469775,RD02,4,2,777.0,2021-04-21 09:20:01


In [9]:
df2.sort_values(by='datetime')

,sysname,str_id,group_v,esm_v,datetime
7535661,HH01,0,2,774.1,2020-01-30 22:23:21
5625023,HH01,1,2,774.4,2020-01-30 22:23:22
7535662,HH01,2,2,774.3,2020-01-30 22:23:22
5625024,HH01,3,2,774.3,2020-01-30 22:23:22
7535663,HH01,4,2,774.4,2020-01-30 22:23:23
...,...,...,...,...,...
3333871,GC01,4,3,833.4,2021-05-17 22:21:02
5622094,GC01,5,3,833.2,2021-05-17 22:21:02
3333872,GC01,6,3,833.7,2021-05-17 22:21:03
5622095,GC01,7,3,833.2,2021-05-17 22:21:03


In [22]:
df3 = df2.groupby(['sysname','str_id','group_v'])
df4 = df3.get_group(('HH01',0, 2))

In [23]:
df3.head(1000)

,sysname,str_id,group_v,esm_v,datetime
0,CW01,2,4,863.1,2020-12-19 02:09:00
1,CW01,4,4,862.8,2020-12-19 02:09:00
2,CW01,6,4,862.6,2020-12-19 02:09:00
3,CW01,0,4,863.5,2020-12-19 02:10:00
4,CW01,2,4,863.8,2020-12-19 02:10:00
...,...,...,...,...,...
12583878,RD02,1,1,709.5,2020-07-14 11:58:01
12583879,RD02,3,1,710.5,2020-07-14 11:58:02
12583880,RD02,0,1,693.2,2020-07-14 11:59:01
12583881,RD02,2,1,692.4,2020-07-14 11:59:01


In [49]:
df4.shape[0]

6172

In [59]:
df4

,sysname,str_id,group_v,esm_v,datetime
7535661,HH01,0,2,774.1,2020-01-30 22:23:21
5625027,HH01,0,2,774.2,2020-01-30 22:24:00
7535670,HH01,0,2,774.3,2020-01-30 22:25:00
5625036,HH01,0,2,774.4,2020-01-30 22:26:00
7535679,HH01,0,2,774.5,2020-01-30 22:27:00
...,...,...,...,...,...
10447947,HH01,0,2,776.0,2021-05-05 14:09:01
10696245,HH01,0,2,768.6,2021-05-15 16:27:00
10696273,HH01,0,2,774.0,2021-05-15 16:34:00
10696545,HH01,0,2,778.5,2021-05-15 17:42:00


In [73]:
df5.datetime.iloc[0]

Timestamp('2020-01-30 22:23:21')

In [ ]:
for i in df4.iterrows:
    if i.

In [41]:
pd.set_option('display.max_rows',1000)


<IPython.core.display.Javascript object>

In [ ]:
df3.head(1)

In [29]:
df3 = df2.groupby(["sysname"]).get_group("GC01")

In [32]:
df3.groupby(["str_id"]).get_group(0)

,sysname,str_id,group_v,esm_v,datetime
3333874,GC01,0,4,856.3,2020-05-19 00:00:00
1048579,GC01,0,4,856.3,2020-05-19 00:01:00
3333883,GC01,0,4,856.3,2020-05-19 00:02:01
1048588,GC01,0,4,856.3,2020-05-19 00:03:01
3333892,GC01,0,4,856.3,2020-05-19 00:04:01
...,...,...,...,...,...
3325481,GC01,0,3,833.3,2021-05-17 22:17:01
5622078,GC01,0,3,833.3,2021-05-17 22:18:01
3333860,GC01,0,3,833.3,2021-05-17 22:19:01
5622087,GC01,0,3,833.0,2021-05-17 22:20:01


In [133]:
df.shape[0]

13469777

In [131]:
start = time.time()
df = df.drop_duplicates()
end = time.time()
print(end - start)

69.31291818618774


In [24]:
start = time.time()
df5 = pd.DataFrame()
df5 = df2[['sysname', 'str_id','group_v','datetime']].groupby(['sysname', 'str_id','group_v']).size().reset_index(name='count')
end = time.time()
print(end - start)

2.2525928020477295


In [25]:
df5

,sysname,str_id,group_v,count
0,CW01,0,3,58
1,CW01,0,4,131256
2,CW01,1,3,65
3,CW01,1,4,155760
4,CW01,2,3,68
5,CW01,2,4,155821
6,CW01,3,3,69
7,CW01,3,4,155783
8,CW01,4,3,74
9,CW01,4,4,155929


In [130]:
df2 = df2.sort_values(by=['str_id','sysname','group_v'])['datetime']

,sysname,str_id,group_v,esm_v,datetime
14661,CW01,0,3,839.9,2020-04-01 17:49:00
14668,CW01,0,3,839.9,2020-04-01 17:51:00
14672,CW01,0,3,839.9,2020-04-01 17:52:00
14679,CW01,0,3,839.9,2020-04-01 17:54:00
14686,CW01,0,3,839.9,2020-04-01 17:56:00
...,...,...,...,...,...
277522,HH01,8,4,851.8,2021-04-01 11:37:03
277531,HH01,8,4,854.7,2021-04-01 11:39:03
277540,HH01,8,4,855.3,2021-04-01 11:41:03
277549,HH01,8,4,855.5,2021-04-01 11:43:03


In [61]:
df5 = df2.groupby(['sysname', 'str_id','group_v']).size().reset_index(name='count')

In [59]:
pd.set_option('display.max_rows',500)
df5

,sysname,str_id,group_v,count
0,CW01,0,3,186
1,CW01,0,4,131128
2,CW01,1,3,4595
3,CW01,1,4,151230
4,CW01,2,3,3739
5,CW01,2,4,152150
6,CW01,3,3,5013
7,CW01,3,4,150839
8,CW01,4,3,4844
9,CW01,4,4,151159


In [68]:
df2.iloc[185].datetime 

sysname                    CW01
str_id                        0
group_v                       3
esm_v                     838.8
datetime    2021-03-03 09:01:00
Name: 219689, dtype: object

In [207]:
df2.groupby(['str_id'])

In [208]:
df2.head()

,sysname,str_id,group_v,esm_v,datetime
0,CW01,2,4,863.1,2020-12-19 02:09:00
1,CW01,4,4,862.8,2020-12-19 02:09:00
2,CW01,6,4,862.6,2020-12-19 02:09:00
3,CW01,0,4,863.5,2020-12-19 02:10:00
4,CW01,2,4,863.8,2020-12-19 02:10:00


In [26]:
df4 = df2.copy()

In [210]:
df4.groupby(['sysname','str_id'])

In [211]:
df4.head()

,sysname,str_id,group_v,esm_v,datetime
0,CW01,2,4,863.1,2020-12-19 02:09:00
1,CW01,4,4,862.8,2020-12-19 02:09:00
2,CW01,6,4,862.6,2020-12-19 02:09:00
3,CW01,0,4,863.5,2020-12-19 02:10:00
4,CW01,2,4,863.8,2020-12-19 02:10:00


# Duration

In [78]:
import datetime
from datetime import timedelta
df3 = df5
total_time0 = timedelta(seconds=0)
total_time1 = timedelta(seconds=0)
total_time2 = timedelta(seconds=0)
total_time3 = timedelta(seconds=0)
total_time4 = timedelta(seconds=0)
start = df3.datetime.iloc[0]
old_group_v = df3.group_v.iloc[0]
for index,row in df3.iterrows():
    if row.group_v!=old_group_v:
        if old_group_v==0:
            total_time0+=(row.datetime - start)
            start = row.datetime
            old_group=row.group_v
        elif old_group_v==1:
            total_time1+=(row.datetime - start)
            start = row.datetime
            old_group=row.group_v
        elif old_group_v==2:
            total_time2=total_time2+ timedelta((row.datetime - start))
            start = row.datetime
            old_group=row.group_v
        elif old_group_v==3:
            total_time3+=(row.datetime - start)
            start = row.datetime
            old_group=row.group_v
        elif old_group_v==4:
            total_time4+=(row.datetime - start)
            start = row.datetime
            old_group=row.group_v
conditions = [ 
        (df3.group_v == 0),
        (df3.group_v==1),
        (df3.group_v == 2),
        (df3.group_v == 3),
        (df3.group_v==4)
    ]
values = [total_time0,total_time1,total_time2,total_time3,total_time4]
df3['duration'] = np.select(conditions,values)
df3.duration

<IPython.core.display.Javascript object>

7535661    0 days
5625027    0 days
7535670    0 days
5625036    0 days
7535679    0 days
            ...  
10447947   0 days
10696245   0 days
10696273   0 days
10696545   0 days
6859573    0 days
Name: duration, Length: 6172, dtype: timedelta64[ns]

In [79]:
df3

,sysname,str_id,group_v,esm_v,datetime,duration
7535661,HH01,0,2,774.1,2020-01-30 22:23:21,0 days
5625027,HH01,0,2,774.2,2020-01-30 22:24:00,0 days
7535670,HH01,0,2,774.3,2020-01-30 22:25:00,0 days
5625036,HH01,0,2,774.4,2020-01-30 22:26:00,0 days
7535679,HH01,0,2,774.5,2020-01-30 22:27:00,0 days
...,...,...,...,...,...,...
10447947,HH01,0,2,776.0,2021-05-05 14:09:01,0 days
10696245,HH01,0,2,768.6,2021-05-15 16:27:00,0 days
10696273,HH01,0,2,774.0,2021-05-15 16:34:00,0 days
10696545,HH01,0,2,778.5,2021-05-15 17:42:00,0 days


In [80]:
df3.tail(900)

,sysname,str_id,group_v,esm_v,datetime,duration
11435469,HH01,0,2,760.1,2021-03-19 13:11:00,0 days
11435473,HH01,0,2,759.9,2021-03-19 13:12:01,0 days
11435477,HH01,0,2,759.5,2021-03-19 13:13:01,0 days
9535006,HH01,0,2,758.3,2021-03-19 13:14:01,0 days
9535010,HH01,0,2,760.8,2021-03-19 13:15:00,0 days
9535014,HH01,0,2,759.7,2021-03-19 13:16:00,0 days
9535018,HH01,0,2,752.2,2021-03-19 13:17:00,0 days
9535022,HH01,0,2,756.7,2021-03-19 13:18:01,0 days
9535026,HH01,0,2,758.2,2021-03-19 13:19:01,0 days
9535030,HH01,0,2,742.4,2021-03-19 13:20:00,0 days


In [30]:
df4['time_difference_sum'] = df4.groupby('str_id')['datetime'].diff()


In [31]:
df4.head(1000)

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum
0,CW01,2,4,863.1,2020-12-19 02:09:00,NaT
1,CW01,4,4,862.8,2020-12-19 02:09:00,NaT
2,CW01,6,4,862.6,2020-12-19 02:09:00,NaT
3,CW01,0,4,863.5,2020-12-19 02:10:00,NaT
4,CW01,2,4,863.8,2020-12-19 02:10:00,0 days 00:01:00
5,CW01,4,4,863.4,2020-12-19 02:10:00,0 days 00:01:00
6,CW01,6,4,863.2,2020-12-19 02:10:00,0 days 00:01:00
7,CW01,0,4,863.2,2020-12-19 02:11:00,0 days 00:01:00
8,CW01,2,4,863.5,2020-12-19 02:11:00,0 days 00:01:00
9,CW01,4,4,863.1,2020-12-19 02:11:00,0 days 00:01:00


In [29]:
from datetime import timedelta

In [149]:
df4['Errors'] = df4.time_difference_sum> timedelta(minutes=5)

In [82]:
df4.head(1000)

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum,Errors
5832,CW01,0,4,849.6,2020-03-31 12:55:00,NaT,False
5839,CW01,0,4,849.5,2020-03-31 12:56:00,0 days 00:01:00,False
5846,CW01,0,4,849.4,2020-03-31 12:57:00,0 days 00:01:00,False
5853,CW01,0,4,850.2,2020-03-31 12:59:00,0 days 00:02:00,False
5860,CW01,0,4,850.3,2020-03-31 13:01:00,0 days 00:02:00,False
5867,CW01,0,4,850.3,2020-03-31 13:03:00,0 days 00:02:00,False
5874,CW01,0,4,850.3,2020-03-31 13:05:00,0 days 00:02:00,False
5881,CW01,0,4,850.3,2020-03-31 13:07:00,0 days 00:02:00,False
5892,CW01,0,4,850.3,2020-03-31 13:10:00,0 days 00:03:00,False
5899,CW01,0,4,850.3,2020-03-31 13:12:00,0 days 00:02:00,False


In [83]:
df4['time_difference_sum'].loc[df4.Errors==True] = timedelta(seconds=0)

C:\Users\abylk\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [84]:
df4.head(1000)

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum,Errors
5832,CW01,0,4,849.6,2020-03-31 12:55:00,NaT,False
5839,CW01,0,4,849.5,2020-03-31 12:56:00,0 days 00:01:00,False
5846,CW01,0,4,849.4,2020-03-31 12:57:00,0 days 00:01:00,False
5853,CW01,0,4,850.2,2020-03-31 12:59:00,0 days 00:02:00,False
5860,CW01,0,4,850.3,2020-03-31 13:01:00,0 days 00:02:00,False
5867,CW01,0,4,850.3,2020-03-31 13:03:00,0 days 00:02:00,False
5874,CW01,0,4,850.3,2020-03-31 13:05:00,0 days 00:02:00,False
5881,CW01,0,4,850.3,2020-03-31 13:07:00,0 days 00:02:00,False
5892,CW01,0,4,850.3,2020-03-31 13:10:00,0 days 00:03:00,False
5899,CW01,0,4,850.3,2020-03-31 13:12:00,0 days 00:02:00,False


In [86]:
df6 = df4.groupby(['sysname','str_id','group_v'], axis=0)["time_difference_sum"].sum()

In [88]:
df6.head(200)

sysname  str_id  group_v
CW01     0       3           0 days 01:59:00
                 4         175 days 06:41:00
         1       3           0 days 02:03:00
                 4         200 days 23:41:00
         2       3           0 days 02:16:00
                 4         200 days 16:33:00
         3       3           0 days 02:12:00
                 4         201 days 00:18:00
         4       3           0 days 02:20:00
                 4         200 days 19:31:00
         5       3           0 days 02:01:00
                 4         201 days 00:40:00
         6       3           0 days 02:07:00
                 4         179 days 17:44:00
GC01     0       2           2 days 13:26:37
                 3          22 days 03:11:10
                 4         329 days 12:51:36
         1       2           2 days 05:17:33
                 3          20 days 15:06:25
                 4         328 days 02:08:35
         2       2           2 days 13:54:42
                 3          22

In [89]:
df6 =df4[df4["time_difference_sum"].dt.total_seconds() < 86400].groupby(['sysname','str_id','group_v'], axis=0)["time_difference_sum"].sum()
df6.head(200)

sysname  str_id  group_v
CW01     0       3           0 days 01:59:00
                 4         175 days 06:41:00
         1       3           0 days 02:03:00
                 4         200 days 23:41:00
         2       3           0 days 02:16:00
                 4         200 days 16:33:00
         3       3           0 days 02:12:00
                 4         201 days 00:18:00
         4       3           0 days 02:20:00
                 4         200 days 19:31:00
         5       3           0 days 02:01:00
                 4         201 days 00:40:00
         6       3           0 days 02:07:00
                 4         179 days 17:44:00
GC01     0       2           2 days 13:26:37
                 3          22 days 03:11:10
                 4         329 days 12:51:36
         1       2           2 days 05:17:33
                 3          20 days 15:06:25
                 4         328 days 02:08:35
         2       2           2 days 13:54:42
                 3          22

In [128]:
df4.groupby(['sysname','str_id','group_v'], axis=0)["time_difference_sum"].count()

sysname  str_id  group_v
CW01     0       3             186
                 4          131127
         1       3            4595
                 4          151229
         2       3            3739
                 4          152149
         3       3            5013
                 4          150838
         4       3            4844
                 4          151158
         5       3            3686
                 4          152137
         6       3            4264
                 4          133603
GC01     0       2            3690
                 3           57677
                 4          448748
         1       2            3202
                 3           54991
                 4          447310
         2       2            3719
                 3           58170
                 4          448149
         3       2            3713
                 3           58163
                 4          448234
         4       2            3699
                 3           5

In [123]:
df4.head()

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum
5832,CW01,0,4,849.6,2020-03-31 12:55:00,NaT
5839,CW01,0,4,849.5,2020-03-31 12:56:00,0 days 00:01:00
5846,CW01,0,4,849.4,2020-03-31 12:57:00,0 days 00:01:00
5853,CW01,0,4,850.2,2020-03-31 12:59:00,0 days 00:02:00
5860,CW01,0,4,850.3,2020-03-31 13:01:00,0 days 00:02:00


In [143]:
df4

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum,Errors
5832,CW01,0,4,849.6,2020-03-31 12:55:00,NaT,False
5839,CW01,0,4,849.5,2020-03-31 12:56:00,0 days 00:01:00,False
5846,CW01,0,4,849.4,2020-03-31 12:57:00,0 days 00:01:00,False
5853,CW01,0,4,850.2,2020-03-31 12:59:00,0 days 00:02:00,False
5860,CW01,0,4,850.3,2020-03-31 13:01:00,0 days 00:02:00,False
...,...,...,...,...,...,...,...
7535644,HH01,8,4,860.8,2021-05-17 22:11:02,0 days 00:01:00,False
7535648,HH01,8,4,861.4,2021-05-17 22:12:03,0 days 00:01:01,False
7535652,HH01,8,4,861.9,2021-05-17 22:13:02,0 days 00:00:59,False
7535656,HH01,8,4,861.9,2021-05-17 22:14:02,0 days 00:01:00,False


In [96]:
df6

sysname  str_id  group_v
CW01     0       3           0 days 01:59:00
                 4         175 days 06:41:00
         1       3           0 days 02:03:00
                 4         200 days 23:41:00
         2       3           0 days 02:16:00
                 4         200 days 16:33:00
         3       3           0 days 02:12:00
                 4         201 days 00:18:00
         4       3           0 days 02:20:00
                 4         200 days 19:31:00
         5       3           0 days 02:01:00
                 4         201 days 00:40:00
         6       3           0 days 02:07:00
                 4         179 days 17:44:00
GC01     0       2           2 days 13:26:37
                 3          22 days 03:11:10
                 4         329 days 12:51:36
         1       2           2 days 05:17:33
                 3          20 days 15:06:25
                 4         328 days 02:08:35
         2       2           2 days 13:54:42
                 3          22

In [129]:
    conditions = [ 
        (df6.time_difference_sum<timedelta(days=5)),
        (df6.time_difference_sum>timedelta(days=5)) & (df6.time_difference_sum<timedelta(days=20)),
        (df6.time_difference_sum>timedelta(days=20))
    ]
    values = ['<5','5-20','>20']
    df6['duration(days)'] = np.select(conditions,values)

In [141]:
df6['count'] = df6.groupby(['duration(days)']).size().reset_index(name='count')

In [142]:
df6

,sysname,str_id,group_v,time_difference_sum,duration(days),count
0,CW01,0,3,0 days 01:59:00,<5,5-20
1,CW01,0,4,175 days 06:41:00,>20,<5
2,CW01,1,3,0 days 02:03:00,<5,>20
3,CW01,1,4,200 days 23:41:00,>20,NaN
4,CW01,2,3,0 days 02:16:00,<5,NaN
5,CW01,2,4,200 days 16:33:00,>20,NaN
6,CW01,3,3,0 days 02:12:00,<5,NaN
7,CW01,3,4,201 days 00:18:00,>20,NaN
8,CW01,4,3,0 days 02:20:00,<5,NaN
9,CW01,4,4,200 days 19:31:00,>20,NaN


In [130]:
df6

,sysname,str_id,group_v,time_difference_sum,duration(days)
0,CW01,0,3,0 days 01:59:00,<5
1,CW01,0,4,175 days 06:41:00,>20
2,CW01,1,3,0 days 02:03:00,<5
3,CW01,1,4,200 days 23:41:00,>20
4,CW01,2,3,0 days 02:16:00,<5
5,CW01,2,4,200 days 16:33:00,>20
6,CW01,3,3,0 days 02:12:00,<5
7,CW01,3,4,201 days 00:18:00,>20
8,CW01,4,3,0 days 02:20:00,<5
9,CW01,4,4,200 days 19:31:00,>20


In [98]:
df4 = df2

In [87]:
pd.set_option('display.max_columns',500)
df

,Unnamed: 0,datetime,comname,sysname,str_id,status,soc,esm_v,curr,mincellv,mincellesm,mincellcell,maxcellv,maxcellesm,maxcellcell,maxtempesm,maxtempsensor,temp0,temp1,temp2,maxtemp,mintemp,avgtemp,esmc_flags,stop0,stop1,stop2
0,1,12/19/2020 2:09,ChunWo,CW01,2,5,95.5,863.1,-0.21,4.042,6,0,4.117,5,0,6,0,29.0,27.0,27.3,29.0,27.0,27.8,15,2,439,65515
1,2,12/19/2020 2:09,ChunWo,CW01,4,517,100.0,862.8,-0.28,4.087,15,10,4.130,10,7,15,0,27.1,26.3,25.9,27.1,25.9,26.4,15,2,436,65508
2,3,12/19/2020 2:09,ChunWo,CW01,6,517,99.7,862.6,-0.24,4.087,7,6,4.113,15,0,14,0,27.6,26.1,26.2,27.6,26.1,26.6,15,2,434,65512
3,4,12/19/2020 2:10,ChunWo,CW01,0,517,99.3,863.5,0.37,4.091,6,2,4.116,1,2,11,0,27.1,26.1,25.9,27.1,25.9,26.4,15,2,443,37
4,5,12/19/2020 2:10,ChunWo,CW01,2,5,95.5,863.8,0.41,4.043,6,4,4.121,7,10,6,0,29.0,27.0,27.3,29.0,27.0,27.8,15,2,446,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97051,1958268,2021-04-21 09:19:01,Gammon,RD02,5,5,64.4,776.6,3.79,3.680,12,13,3.705,4,6,5,0,29.0,28.7,28.7,29.0,28.7,28.8,15,7766,379,98
97052,1958269,2021-04-21 09:20:00,Gammon,RD02,0,517,64.4,776.3,2.72,3.686,15,5,3.705,2,8,12,0,29.2,28.7,28.6,29.2,28.6,28.8,15,7763,272,187
97053,1958270,2021-04-21 09:20:01,Gammon,RD02,2,5,64.9,776.4,2.54,3.685,2,1,3.706,6,6,6,0,29.1,28.8,29.0,29.1,28.8,29.0,15,7764,254,126
97054,1958271,2021-04-21 09:20:01,Gammon,RD02,4,517,44.7,777.0,2.59,3.612,5,13,3.726,4,6,10,0,29.6,28.2,28.5,29.6,28.2,28.8,15,7770,259,118


# PySpark trial

In [16]:
pip install findspark

In [100]:
df6

sysname  str_id  group_v
CW01     0       3           0 days 01:59:00
                 4         175 days 06:41:00
         1       3           0 days 02:03:00
                 4         200 days 23:41:00
         2       3           0 days 02:16:00
                 4         200 days 16:33:00
         3       3           0 days 02:12:00
                 4         201 days 00:18:00
         4       3           0 days 02:20:00
                 4         200 days 19:31:00
         5       3           0 days 02:01:00
                 4         201 days 00:40:00
         6       3           0 days 02:07:00
                 4         179 days 17:44:00
GC01     0       2           2 days 13:26:37
                 3          22 days 03:11:10
                 4         329 days 12:51:36
         1       2           2 days 05:17:33
                 3          20 days 15:06:25
                 4         328 days 02:08:35
         2       2           2 days 13:54:42
                 3          22

In [102]:
df4

,sysname,str_id,group_v,esm_v,datetime,time_difference_sum,Errors
5832,CW01,0,4,849.6,2020-03-31 12:55:00,NaT,False
5839,CW01,0,4,849.5,2020-03-31 12:56:00,0 days 00:01:00,False
5846,CW01,0,4,849.4,2020-03-31 12:57:00,0 days 00:01:00,False
5853,CW01,0,4,850.2,2020-03-31 12:59:00,0 days 00:02:00,False
5860,CW01,0,4,850.3,2020-03-31 13:01:00,0 days 00:02:00,False
...,...,...,...,...,...,...,...
7535644,HH01,8,4,860.8,2021-05-17 22:11:02,0 days 00:01:00,False
7535648,HH01,8,4,861.4,2021-05-17 22:12:03,0 days 00:01:01,False
7535652,HH01,8,4,861.9,2021-05-17 22:13:02,0 days 00:00:59,False
7535656,HH01,8,4,861.9,2021-05-17 22:14:02,0 days 00:01:00,False


In [114]:
df9 = df4.copy()

In [115]:
df9 = df9.drop(['Errors','time_difference_sum','esm_v','datetime'],axis=1)

In [116]:
df9 = df9.groupby(['sysname','str_id'])

In [125]:
df6 = df6.reset_index()

In [ ]:
df6 =